In [1]:
import mmh3

In [2]:
import functools

In [3]:
import iteration_utilities

In [4]:
import random

In [5]:
import numpy as np

In [6]:
import math

In [7]:
import tensorflow as tf

In [8]:
from tensorflow.python import debug as tf_debug

In [9]:
from enum import IntEnum

In [10]:
print("TensorFlow version: {}".format(tf.VERSION))

TensorFlow version: 1.8.0


In [11]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
training_data, testing_data = tf.keras.datasets.fashion_mnist.load_data()

In [12]:
number_of_classes = int(np.max(testing_data[1])) + 1
number_of_input_channels = 1

In [13]:
number_of_experts = 1

In [14]:
number_of_hidden_layers = 20

In [15]:
number_of_features_per_layer = 32

In [16]:
paddings = (0, 0), (2, 2), (2, 2)

In [17]:
random_seed = number_of_hidden_layers

In [18]:
def offset_initializer():
    return tf.random_normal_initializer(stddev=1.0)
#     return tf.random_uniform_initializer(minval=-3.0, maxval=3.0)

In [19]:
def grid_sample(images, offset_x, offset_y):
    tf.assert_rank(images, 4)
    tf.assert_rank(offset_x, 4)
    tf.assert_rank(offset_y, 4)
    image_shape = tf.shape(images)
    current_batch_size = image_shape[0]
    width = image_shape[1]
    height = image_shape[2]
    number_of_features = image_shape[3]
    
    def check_width(tensor):
        if (tf.test.is_gpu_available()):
            return tensor
        else:
            return tf.maximum(tf.minimum(tensor, width - 1), 0)
    
    def check_height(tensor):
        if (tf.test.is_gpu_available()):
            return tensor
        else:
            return tf.maximum(tf.minimum(tensor, height - 1), 0)
    
    offset_left = tf.floor(offset_x)
    offset_top = tf.floor(offset_y)
    tf.assert_equal(tf.shape(offset_left), image_shape)
    tf.assert_equal(tf.shape(offset_top), image_shape)

    
    eps = 1e-7

    factor_right = offset_x - offset_left + eps
    factor_left = 1.0 + 2.0 * eps - factor_right
    factor_bottom = offset_y - offset_top + eps
    factor_top = 1.0 + 2.0 * eps - factor_bottom
    
    image_index, x_index, y_index, feature_index = tf.meshgrid(
        tf.range(current_batch_size, dtype=tf.int32),
        tf.range(width, dtype=tf.int32),
        tf.range(height, dtype=tf.int32),
        tf.range(number_of_features, dtype=tf.int32),
        indexing='ij',
    )

    left_index = check_width(x_index + tf.cast(offset_left, tf.int32))
    top_index = check_height(y_index + tf.cast(offset_top, tf.int32))

    right_index = left_index + 1
    bottom_index = top_index + 1
    
    tf.assert_rank(left_index, 4)
    tf.assert_rank(right_index, 4)
    tf.assert_rank(bottom_index, 4)
    tf.assert_rank(top_index, 4)
    
    images_top_left = tf.gather_nd(images, tf.stack((image_index, left_index, top_index, feature_index), axis=4))
    images_top_right = tf.gather_nd(images, tf.stack((image_index, right_index, top_index, feature_index), axis=4))
    images_bottom_left = tf.gather_nd(images, tf.stack((image_index, left_index, bottom_index, feature_index), axis=4))
    images_bottom_right = tf.gather_nd(images, tf.stack((image_index, right_index, bottom_index, feature_index), axis=4))

    tf.assert_rank(images_top_left, 4)
    tf.assert_rank(images_top_right, 4)
    tf.assert_rank(images_bottom_left, 4)
    tf.assert_rank(images_bottom_right, 4)

    lerp_top = factor_left * images_top_left + factor_right * images_top_right
    lerp_bottom = factor_left * images_bottom_left + factor_right * images_bottom_right
    output = factor_top * lerp_top + factor_bottom * lerp_bottom
    tf.assert_rank(output, 4)
    return output

In [20]:
def offnet(images, number_of_features, number_of_experts=None, name=None):
    with tf.variable_scope(name, default_name="offnet"):
        input_shape = images.shape
        if number_of_experts is None:
            number_of_experts = int(input_shape[0])
        number_of_input_features = int(input_shape[4])
        score_weight = tf.get_variable(
            name="score_weight",
            initializer=tf.random_normal_initializer(stddev=tf.sqrt(1.0 / number_of_input_features)),
            dtype=tf.float32,
            shape=(number_of_experts, number_of_input_features, number_of_features)
        )
        offset_x_weight = tf.get_variable(
            name="offset_x_weight",
            initializer=tf.random_normal_initializer(stddev=tf.sqrt(1.0 / number_of_input_features)),
            dtype=tf.float32,
            shape=(number_of_experts, number_of_input_features, number_of_features)
        )
        offset_y_weight = tf.get_variable(
            name="offset_y_weight",
            initializer=tf.random_normal_initializer(stddev=tf.sqrt(1.0 / number_of_input_features)),
            dtype=tf.float32,
            shape=(number_of_experts, number_of_input_features, number_of_features)
        )
        offset_x_bias = tf.get_variable(
            name="offset_x_bias",
            initializer=offset_initializer(),
            dtype=tf.float32,
            shape=(number_of_experts, 1, number_of_features),
        )
        offset_y_bias = tf.get_variable(
            name="offset_y_bias",
            initializer=offset_initializer(),
            dtype=tf.float32,
            shape=(number_of_experts, 1,  number_of_features),
        )
#         def scale_initializer():
#             return tf.sqrt(weight_scale / number_of_input_features)
#         score_scale = tf.get_variable(
#             name="score_scale",
#             initializer=scale_initializer(),
#             dtype=tf.float32
#         )
#         offset_x_scale = tf.get_variable(
#             name="offset_x_scale",
#             initializer=scale_initializer(),
#             dtype=tf.float32
#         )
#         offset_y_scale = tf.get_variable(
#             name="offset_y_scale",
#             initializer=scale_initializer(),
#             dtype=tf.float32
#         )
        
        dynamic_image_shape = tf.shape(images)
        images_3d = tf.reshape(
            images,
            (
                number_of_experts,
                dynamic_image_shape[1] * dynamic_image_shape[2] * dynamic_image_shape[3],
                number_of_input_features
            )
        )
        def images_xw_plus_b(w, b):
            tf.matmul(images_3d, w) + b
        
        def to_4d(images):
            tf.assert_equal(tf.shape(images), (number_of_experts, dynamic_image_shape[1] * dynamic_image_shape[2] * dynamic_image_shape[3], number_of_features))
            return tf.reshape(
                images,
                (
                    number_of_experts * dynamic_image_shape[1],
                    dynamic_image_shape[2],
                    dynamic_image_shape[3],
                    number_of_features
                )
            )
        return tf.reshape(
            grid_sample(
                to_4d(tf.matmul(images_3d, score_weight)),
                to_4d(tf.matmul(images_3d, offset_x_weight) + offset_x_bias),
                to_4d(tf.matmul(images_3d, offset_y_weight) + offset_y_bias)
            ),
            (
                number_of_experts,
                dynamic_image_shape[1],
                dynamic_image_shape[2],
                dynamic_image_shape[3],
                number_of_features
            )
        )        

In [21]:
number_of_horizontal_samples = 3
number_of_vertical_samples = 3

In [22]:
def model_fn(features, labels, mode, params, config):
    random.seed(number_of_hidden_layers)
    padded_input = tf.expand_dims(
        tf.expand_dims(
            tf.pad(tf.cast(features, tf.float32) / 255.0 - 0.5, paddings),
            axis=0
        ),
        axis=4
    )
    width = int(padded_input.shape[2])
    height = int(padded_input.shape[3])
    
    layers = functools.reduce(
        lambda features, i: tf.nn.relu(tf.layers.batch_normalization(
            offnet(features, number_of_features_per_layer, 1),
            training=mode is tf.estimator.ModeKeys.TRAIN
        )),
        range(number_of_hidden_layers),
        padded_input
    )
    score_layer = offnet(layers, number_of_classes, number_of_experts)
    scores = tf.reduce_mean(
        tf.reduce_mean(
            score_layer,
            axis=0
        )[
            :,
            int(width / (number_of_horizontal_samples + 1)):int(width * number_of_horizontal_samples / (number_of_horizontal_samples + 1)):int(width / (number_of_horizontal_samples + 1)),
            int(height / (number_of_horizontal_samples + 1)):int(height * number_of_horizontal_samples / (number_of_horizontal_samples + 1)):int(height / (number_of_horizontal_samples + 1)),
            :
        ],
        axis=(1, 2)
    )

    probabilities = tf.nn.softmax(logits=scores)
    predicted_classes = tf.argmax(scores, 1)
    predictions = {
        'probabilities' : probabilities,
        'scores': scores,
        'class': predicted_classes,
    }

    optimizer = tf.train.AdamOptimizer(epsilon=0.1)
    if labels is None:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions,
        )
    else:
        loss = tf.losses.softmax_cross_entropy(logits=scores, onehot_labels=tf.one_hot(labels, number_of_classes))
        eval_metric_ops = {
            'accuracy': tf.metrics.accuracy(labels=labels, predictions=predicted_classes)
        }
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            return tf.estimator.EstimatorSpec(
                mode=mode,
                predictions=predictions,
                loss=loss,
                train_op=optimizer.minimize(loss, global_step=tf.train.get_global_step()),
                eval_metric_ops=eval_metric_ops,
            )

In [23]:
run_config = tf.estimator.RunConfig(
    model_dir=f"models/simplereluoffnet{number_of_features_per_layer}x{number_of_hidden_layers}",
    session_config=tf.ConfigProto(
        gpu_options=tf.GPUOptions(
            allow_growth=True,
        ),
        graph_options=tf.GraphOptions(
            optimizer_options=tf.OptimizerOptions(
                global_jit_level=tf.OptimizerOptions.ON_2,
                do_function_inlining=True,
                do_constant_folding=True,
                do_common_subexpression_elimination=True,
            ),
        )
    )
)

In [24]:
estimator = tf.estimator.Estimator(model_fn, config=run_config)

INFO:tensorflow:Using config: {'_model_dir': 'models/simplereluoffnet32x20', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  allow_growth: true
}
graph_options {
  optimizer_options {
    do_common_subexpression_elimination: true
    do_constant_folding: true
    do_function_inlining: true
    global_jit_level: ON_2
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff4b56bf320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
batch_size = 32

In [26]:
def training_dataset():
    return tf.data.Dataset.from_tensor_slices(training_data).shuffle(1000).batch(batch_size)

In [27]:
def testing_dataset():
    return tf.data.Dataset.from_tensor_slices(testing_data).batch(batch_size)

In [28]:
# hook = tf_debug.TensorBoardDebugHook("localhost:6064")
# estimator.train(training_dataset,hooks=[hook])

In [ ]:
tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(training_dataset),
    eval_spec=tf.estimator.EvalSpec(testing_dataset, throttle_secs=1800)
)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 1800 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into models/simplereluoffnet32x20/model.ckpt.
INFO:tensorflow:loss = 2.2942164, step = 0
INFO:tensorflow:global_step/sec: 1.10784
INFO:tensorflow:loss = 2.3278217, step = 100 (90.267 sec)
INFO:tensorflow:global_step/sec: 1.20797
INFO:tensorflow:loss = 2.251985, step = 200 (82.783 sec)
INFO:tensorflow:global_step/sec: 1.20618
INFO:tensorflow:loss = 2.2821088, step = 300 (82.907 sec)
INFO:tensorflow:global_step/sec: 1.20517
INFO:tensorflow:loss = 2.333963, step = 400 (82.976 sec)
INFO:tensorflow:global_st

INFO:tensorflow:global_step/sec: 1.18728
INFO:tensorflow:loss = 0.9415648, step = 4550 (84.227 sec)
INFO:tensorflow:global_step/sec: 1.20581
INFO:tensorflow:loss = 0.58358943, step = 4650 (82.932 sec)
INFO:tensorflow:global_step/sec: 1.20558
INFO:tensorflow:loss = 0.68224823, step = 4750 (82.948 sec)
INFO:tensorflow:global_step/sec: 1.20272
INFO:tensorflow:loss = 0.5464735, step = 4850 (83.145 sec)
INFO:tensorflow:global_step/sec: 1.20463
INFO:tensorflow:loss = 0.4329256, step = 4950 (83.013 sec)
INFO:tensorflow:global_step/sec: 1.20279
INFO:tensorflow:loss = 0.5665639, step = 5050 (83.140 sec)
INFO:tensorflow:global_step/sec: 1.20423
INFO:tensorflow:loss = 0.6380728, step = 5150 (83.041 sec)
INFO:tensorflow:Saving checkpoints for 5186 into models/simplereluoffnet32x20/model.ckpt.
INFO:tensorflow:global_step/sec: 1.18402
INFO:tensorflow:loss = 0.31238797, step = 5250 (84.458 sec)
INFO:tensorflow:global_step/sec: 1.20459
INFO:tensorflow:loss = 0.6957886, step = 5350 (83.016 sec)
INFO:te

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-19-03:54:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/simplereluoffnet32x20/model.ckpt-9375
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-06-19-03:55:54
INFO:tensorflow:Saving dict for global step 9375: accuracy = 0.8315625, global_step = 9375, loss = 0.46885037
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflo

INFO:tensorflow:loss = 0.58314407, step = 13125
INFO:tensorflow:global_step/sec: 1.1056
INFO:tensorflow:loss = 0.45472068, step = 13225 (90.449 sec)
INFO:tensorflow:global_step/sec: 1.20376
INFO:tensorflow:loss = 0.5854492, step = 13325 (83.073 sec)
INFO:tensorflow:global_step/sec: 1.20681
INFO:tensorflow:loss = 0.46972716, step = 13425 (82.863 sec)
INFO:tensorflow:global_step/sec: 1.20303
INFO:tensorflow:loss = 0.42685694, step = 13525 (83.124 sec)
INFO:tensorflow:global_step/sec: 1.20322
INFO:tensorflow:loss = 0.22438553, step = 13625 (83.110 sec)
INFO:tensorflow:global_step/sec: 1.20272
INFO:tensorflow:loss = 0.40254295, step = 13725 (83.145 sec)
INFO:tensorflow:global_step/sec: 1.20653
INFO:tensorflow:loss = 0.32394296, step = 13825 (82.882 sec)
INFO:tensorflow:Saving checkpoints for 13838 into models/simplereluoffnet32x20/model.ckpt.
INFO:tensorflow:global_step/sec: 1.18729
INFO:tensorflow:loss = 0.3060173, step = 13925 (84.226 sec)
INFO:tensorflow:global_step/sec: 1.20571
INFO:te

INFO:tensorflow:global_step/sec: 1.20667
INFO:tensorflow:loss = 0.24905387, step = 18075 (82.873 sec)
INFO:tensorflow:global_step/sec: 1.206
INFO:tensorflow:loss = 0.34800768, step = 18175 (82.919 sec)
INFO:tensorflow:global_step/sec: 1.2041
INFO:tensorflow:loss = 0.37502474, step = 18275 (83.050 sec)
INFO:tensorflow:Saving checkpoints for 18311 into models/simplereluoffnet32x20/model.ckpt.
INFO:tensorflow:global_step/sec: 1.18775
INFO:tensorflow:loss = 0.38844183, step = 18375 (84.193 sec)
INFO:tensorflow:global_step/sec: 1.20669
INFO:tensorflow:loss = 0.47562164, step = 18475 (82.872 sec)
INFO:tensorflow:global_step/sec: 1.20781
INFO:tensorflow:loss = 0.34594846, step = 18575 (82.794 sec)
INFO:tensorflow:global_step/sec: 1.20632
INFO:tensorflow:loss = 0.29872674, step = 18675 (82.897 sec)
INFO:tensorflow:Saving checkpoints for 18750 into models/simplereluoffnet32x20/model.ckpt.
INFO:tensorflow:Loss for final step: 0.32663074.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done cal

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-06-19-07:14:21
INFO:tensorflow:Saving dict for global step 22500: accuracy = 0.86125, global_step = 22500, loss = 0.3649759
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/simplereluoffnet32x20/model.ckpt-22500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 22501 into models/simplereluoffnet32x20/model.ckpt.
INFO:tensor

INFO:tensorflow:global_step/sec: 1.20596
INFO:tensorflow:loss = 0.22703688, step = 26650 (82.922 sec)
INFO:tensorflow:global_step/sec: 1.20663
INFO:tensorflow:loss = 0.19690187, step = 26750 (82.875 sec)
INFO:tensorflow:global_step/sec: 1.20739
INFO:tensorflow:loss = 0.4541081, step = 26850 (82.823 sec)
INFO:tensorflow:global_step/sec: 1.20254
INFO:tensorflow:loss = 0.25210887, step = 26950 (83.157 sec)
INFO:tensorflow:Saving checkpoints for 26964 into models/simplereluoffnet32x20/model.ckpt.
INFO:tensorflow:global_step/sec: 1.18779
INFO:tensorflow:loss = 0.46704584, step = 27050 (84.190 sec)
INFO:tensorflow:global_step/sec: 1.20591
INFO:tensorflow:loss = 0.46008676, step = 27150 (82.925 sec)
INFO:tensorflow:global_step/sec: 1.20569
INFO:tensorflow:loss = 0.4692116, step = 27250 (82.940 sec)
INFO:tensorflow:global_step/sec: 1.20442
INFO:tensorflow:loss = 0.41498604, step = 27350 (83.028 sec)
INFO:tensorflow:global_step/sec: 1.20655
INFO:tensorflow:loss = 0.33217582, step = 27450 (82.88

INFO:tensorflow:global_step/sec: 1.18472
INFO:tensorflow:loss = 0.16315766, step = 31500 (84.408 sec)
INFO:tensorflow:global_step/sec: 1.20577
INFO:tensorflow:loss = 0.2216576, step = 31600 (82.935 sec)
INFO:tensorflow:global_step/sec: 1.2049
INFO:tensorflow:loss = 0.23107074, step = 31700 (82.994 sec)
INFO:tensorflow:global_step/sec: 1.20542
INFO:tensorflow:loss = 0.2532777, step = 31800 (82.959 sec)
INFO:tensorflow:Saving checkpoints for 31875 into models/simplereluoffnet32x20/model.ckpt.
INFO:tensorflow:Loss for final step: 0.35196677.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-19-09:34:47
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/simplereluoffnet32x20/model.ckpt-31875
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:te

In [ ]:
# estimator.evaluate(input_fn=lambda:tf.data.Dataset.from_tensor_slices(testing_data).batch(3).take(1))

In [ ]:
# tuple(estimator.predict(input_fn=lambda:tf.data.Dataset.from_tensor_slices(testing_data).batch(3).take(1)))

In [ ]:
# estimator.train(input_fn=training_dataset, steps=1)